# Informatio Visualization: Project 1
*David G.*
*Pau A.*
*GCED, Course 2023-24*

This Project aims to apply the knowledge adquired during the first two months of the Information Visualization Course at UPC. With this objective in mind, we have applied different visualizations and data processing techniques in order to analyze the behaviour of traffic accidents in NY City. Our goal has been to create a static visualization which allowed us to answer the following questions:
- Are accidents more frequent during weekdays or weekend? Is there any difference between before COVID-19 and after?
- Is there any type of vehicle more prone to participate in accidents
- At what time of the day are accidents more common?
- Are there any areas with a larger number of accidents?
- Is there a correlation between weather conditions and accidents?
  
With there questions in mind, we have performed a diverse set of preprocessing and data transformation techniques to the provided dataset in order to obtain the appropiate visualizations for each of each of the objectives. This notebook aims to guide the reader through the steps that have been followed and the mental process through which the visualizations have been refined and obtained.

The report has the following outline:
- [Data Preprocessing](#data-preprocessing)
- [Desig Process](#design-process)
  - [Question 1](#question-1)
  - [Question 2](#question-2)
  - [Question 3](#question-3)
  - [Question 4](#question-4)
  - [Question 5](#question-5)
- [Results](#results)



Before proceding with the preprocessing and visualization-making process, it is necessary to import the following libraries. In case any of them is not installed, the command to install them is also provided

**IMPORTANT** if running on Google Colab enable `colab` render

In [1]:
# alt.rendered.enable('colab')

In [2]:
# !pip install -r requirements.txt

In [3]:
import altair as alt
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import geodatasets
import h3pandas

# we disable max_rows in altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Data Preprocessing
To begin with, the original dataset has been filtered to encompass the accidents encompassed within the period of *Jun-Sep 2018 and 2019*. This step was performed before downloading the dataset. Afterward, the **Open Refine** software has been used to first approach the data and do the bulk of the preprocessing.

Through Open Refine, we have been able to take a first look at our dataset and determine its columns and which issues need to be solved before proceeding. We identified three main issues:
- Many different encodings are used to refer to the same category on some variables, such as vehicle type
- Columns containg Null Values
- Columns not relevant to our analysis

After further analysis, it was found that the presence of null values was not as important as expected, as the columns that, in our opinion, were the most relevant, did not include many null values. 

On the other hand, the selection of the relevant columns was not performed on Open-Refine, and no columns were removed. This task was later 
performed using Pandas in Python.

However, the main issue with the dataset was the fact that some of the categorical columns were very inconsistent in naming, e.g., the same category was written in many different ways. This was mainly observed in the Vehicle Type column. In other columns, it was not considered relevant, and no action was taken.

On the other hand, a lot of work was directed toward obtaining a more consistent naming scheme for the Vehicle Type 1 column. This was aimed at normalizing the vehicle type codes. In order to do so, the **clustering** methods of Open Refine were used. With this, we were able to mainly fix orthographic inconsistencies. However, we were still left with many different categories or subcategories, which we believed were too granular for our analysis. Therefore, we manually integrated some of the less frequent categories into more general ones.

It is worth noting that answering some of the questions required further preprocessing, which will be explained later.


In [9]:

df = pd.read_csv("dataset_v1.csv").sample(1000)[["CRASH DATE","CRASH TIME","BOROUGH", "LATITUDE", "LONGITUDE","VEHICLE TYPE CODE 1"]]

# We parse the date column as a date
df['date'] = pd.to_datetime(df['CRASH DATE'], format='%Y-%m-%d')                 
print(df.shape)
df.head()


(1000, 7)


,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,VEHICLE TYPE CODE 1,date
77040,2018-07-23T00:00:00Z,15:28,BROOKLYN,40.647526,-73.978195,Sedan,2018-07-23 00:00:00+00:00
3610,2020-09-23T00:00:00Z,8:40,MANHATTAN,40.804520,-73.937790,SUV,2020-09-23 00:00:00+00:00
105145,2018-07-01T00:00:00Z,0:20,BROOKLYN,40.693405,-73.908860,SUV,2018-07-01 00:00:00+00:00
98986,2018-06-24T00:00:00Z,15:00,BROOKLYN,40.619440,-73.999880,SUV,2018-06-24 00:00:00+00:00
6774,2020-09-28T00:00:00Z,0:01,NaN,40.827812,-73.925934,Sedan,2020-09-28 00:00:00+00:00


## Design Process
In order to obtain a visualization which allows us to answer the proposed questions, we decided to adress each question separately and later integrate them all into a single view using streamlit. This section describes the process followed to obtain the visualizations for each question. Afterwards, the results sections presents the final visualizations and the conclusions obtained from them.

### Question 1 
**Are accidents more frequent during weekdays or weekends? Is there any difference between before COVID-19 and after**
We decided that we wanted to obtain a visualization which showed the amount of accidents while allowing the reader to easily compare the difference between both weekday/weekend and before/after covid. 

Therefore, we needed a visualization which allowed to both present a quantity and allow for easy comparisons depending on groupings. We decided that a focus needed to be made on facilitating comparisons of the trend, more than on showing precise information about the distribution of the accidents.

To begin with, we plotted a groupped barplot in which we plotted the number of accidents per day of the week, groupped by before or after covid. We decided that encoding seven distinct values (and therefore bars) was not necessary and could distract the reader. Furthermore, the differences between weekdays were not large. Therefore we decided to plot only for weekday and weekend.

In consequence, we repeated the groupped bar plot calculating the mean of weekday and weekend day accidents and again groupped depending on before or after covid.

At this step we also asked ourselves wether we should group by weekend or covid. As shown in the following graphs, we tried both.

In [5]:
# create coolumn weekday/weekend
df['weekday'] = df['date'].dt.dayofweek 
# We create a column that says wether before or after covid
df['covid'] = df['date'].dt.year
# give name
df['covid'] = df['covid'].replace([2018,2020],['before','after'])

alt.Chart(df[["covid","weekday","date"]]).mark_bar().transform_aggregate(
    accidents = 'count()',
    groupby=['weekday','date','covid']
).encode(
    x='weekday:N',
    y='average(accidents):Q',
    color='weekday',
    column = "covid:N"
)


alt.Chart(...)

We decided to keep the following graph

In [6]:

df['weekday'] = df['weekday'].replace([0,1,2,3,4,5,6],['weekday','weekday','weekday','weekday','weekday','weekend','weekend'])

alt.Chart(df[["covid","weekday","date"]]).mark_bar().transform_aggregate(
    accidents = 'count()',
    groupby=['weekday','date','covid']
).encode(
    x='weekday:N',
    y='average(accidents):Q',
    color='weekday',
    column = "covid:N"
).properties(width=300, title = "Average daily accidents before and after covid")


alt.Chart(...)

It is worth noting, that as shown next, we also tried a boxplot. Even though it shows more description about the data ditribution, we feel that the added complexity is not necessary and the simpler bar graph is preferible.

In [ ]:

alt.Chart(df[]).mark_boxplot().transform_aggregate(
    accidents = 'count()',
    groupby=['weekday','date','covid']
).encode(
    x=alt.X('weekday:N',title=None),
    y=alt.Y('average(accidents):Q'),
    color=alt.Color('weekday'),
    column = alt.Column("covid:N",title=None)
).properties(
    width=200
)

### Question 2

### Question 3
**Accidenys by time of day**
- Vam problar un polar barplot - no permet comparar bé hores diferents i no es escala, no va be

In [ ]:
df_test = df.copy()
# Assuming you have a DataFrame named 'accidents_df' with a 'crash_time' column containing timestamps
# Ensure 'crash_time' column is in datetime format
df_test['crash_time'] = pd.to_datetime(df['CRASH TIME'])

# Create a range of time for 24 hours in 5-minute intervals
start_time = pd.to_datetime('00:00:00')  # Start time
end_time = pd.to_datetime('23:59:59')  # End time

# Create a range of 5-minute intervals for a day
five_min_intervals = pd.date_range(start=start_time, end=end_time, freq='5T')

# Initialize a DataFrame to store counts for each interval
accidents_count = pd.DataFrame(index=five_min_intervals, columns=['accident_count'])
accidents_count['accident_count'] = 0  # Initialize counts to zero

# Count accidents within each interval
for index, row in df_test.iterrows():
    accident_time = row['crash_time']
    # Find the interval to increment the count
    interval = accidents_count.index.searchsorted(accident_time)
    accidents_count.iloc[interval - 1]['accident_count'] += 1


### Question 4
**Are there any areas with a larger number of accidents?**

This question proved challening to solve both in the visualization and technical aspect. To begin with, we made a few hand-drawn prototypes of how we wanted the visualization to look, which made us realize that, given the high density of accidents compared to the area, plotting all the accidents as dots was not a good option.

Therefore, we decided to try a cloropeth but with a higher granularity than buroughs, e.g. precints. This proved technically difficult as we had to learn how to work with geopandas dataframes in order to perform a spatial join between our dataset and a map. The result is shown in the following visualization.

At first we encountered one issue related to which variable we encoded using colour. If we encoded just the number of accidents perpolygon it made smaller polygons appear with a lighter color even if the area had a higher concentration of accidents. To solve it we decided to normalize the count by the area of the polygon. We encoded this new variable using a quantitative sequential color scale.

As can be seen, the result was quie good. Nevertheless, we felt that it would be better to have all the area divisions have a similar size to facilitate comparisons and allow for a more detailed view in some areas of the city.

*(see next cells for the continuation)*

In [ ]:
map_orig = gpd.read_file(gplt.datasets.get_path('nyc_parking_tickets')) 
df_coord = df.dropna(subset=['LATITUDE', 'LONGITUDE'])
gdf = gpd.GeoDataFrame(
    df_coord, geometry=gpd.points_from_xy( df_coord.LONGITUDE,df_coord.LATITUDE)
)[["geometry"]]
gdf.set_crs(epsg=4326, inplace=True)
if gdf.crs != map_orig.crs:
    gdf.to_crs(map_orig.crs, inplace=True)
# join espacially
gdf=gpd.sjoin(gdf,map_orig, how="right", op='intersects')
gdf_count = gdf.groupby(['geometry','id']).size().reset_index(name='counts')
gdf_count["counts"] = gdf_count.apply(lambda row: row["counts"]/(row["geometry"].area), axis=1)
df_geo = pd.DataFrame(gdf_count[["id","counts"]])

alt.Chart(map_orig).mark_geoshape().encode(
    color = 'counts:Q',
    tooltip=['id:N','counts:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_geo, 'id', ['counts'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


In order to solve the aforementioned issues, we decided to make a hexagon style map, which transform the shape of the geography into hexagons and colors them as a choropleth. To do so we first transformed our original map into hexagons, using the *h3pandas* library and the instructions while researching how to do it ([link](https://python.plainenglish.io/creating-beautiful-hexagon-maps-with-python-25c9291eeeda)). We first converted the polygons into hexagons and then repeated the process as in the previous graph.

However, we felt that transforming into hexagons made it more difficult to recognize the geographical areas of the city. Therefore we added clear borders between buroughs and labeled them.


In [ ]:
path = geodatasets.get_path("nybb")
ny = gpd.read_file(path).to_crs("EPSG:4326")
resolution = 8
hex_map = ny.h3.polyfill_resample(resolution)
hex_map=hex_map.to_crs("ESRI:102003")

hex_buroughs = hex_map.copy()
# unite all hexagons in the same burough using boroname
hex_buroughs = hex_buroughs.dissolve(by='BoroName')

hex_buroughs.head()
ny_df=pd.DataFrame()
# get dataframe centroid x,y, burough name
ny_df['x'] = hex_buroughs.centroid.x
ny_df['y'] = hex_buroughs.centroid.y
ny_df["BoroName"] = hex_buroughs.index


df_coord = df.dropna(subset=['LATITUDE', 'LONGITUDE'])
gdf = gpd.GeoDataFrame(
    df_coord, geometry=gpd.points_from_xy( df_coord.LONGITUDE,df_coord.LATITUDE)
)[["geometry"]]
gdf=gdf.set_crs(epsg=4326, inplace=True).to_crs("ESRI:102003")
# give id to each row in gdf from 1 to n

gdf=gpd.sjoin(gdf,hex_map, how="right", op='intersects')
print(gdf.columns)
gdf_count = gdf.groupby(['geometry','h3_polyfill']).size().reset_index(name='counts')
# set index id
gdf_count["counts"] = gdf_count.apply(lambda row: row["counts"], axis=1)
df_geo = pd.DataFrame(gdf_count[["h3_polyfill","counts"]])

hex = hex_map.merge(df_geo, left_on='h3_polyfill', right_on='h3_polyfill', how='left')
# to 0 counts sum 1
hex["counts"]=hex["counts"].apply(lambda x: 1 if x ==0 else x)



In [ ]:
hexagons=alt.Chart(hex).mark_geoshape().encode(
    color = 'counts:Q',
    tooltip=['h3_polyfill:N','counts:Q']
).project(
    type='identity',
    reflectY=True
).properties(
    width=500,
    height=300
)
labels = alt.Chart(ny_df).mark_text().encode(
    longitude='x:Q',
    latitude='y:Q',
    text='BoroName:N'
)
borders = alt.Chart(hex_buroughs).mark_geoshape(
    stroke='darkgray',
    strokeWidth=1.25,
    opacity=1,
    fillOpacity=0
).project(
    type='identity',
    reflectY=True
).properties(
    width=500,
    height=300
)
hexagons + labels+borders

### Question 5
**Is there a correlation between weather conditions and accidents?**

This question proved the most difficult of them all.  To begin with, we had to find a weather dataset. This task was quickly achieved, as many weather datasets are available on the net. However, the first one we used did not provide enough information as many of the columns of the dataset contained many null values. 
Therefore, we decided on using a second dataset, which contains daily weather data from New York City. The dataset contains many different columns, out of which the following ones were considered to be relevant:
- `datetime`
- `precip`
- `temperature`
- `windspeed`
- `visibility`
- `conditions`
  
To begin with, we started by visualizing the distribution of the data using `pandas` histogram and describe functions. We observed that, as expected due to being summer data, the weather had moslty relatively high temperatures with relatively few precipitations. This made the visualization design process more challenging, as we had to alse take it into account.

We tried two approaches; a lollipop chart that just showed the difference in the mean number of accidents depending on the weather and also a set of heatmaps which showed various weather variables coloured by the amount of accidents normalized by the number of days with the weather combintion.

Both approaches are displayed in the following cells. The lollipop chart was chosen and is further explained in the results section.

On the other hand the heatmaps did not achieve good results as many of the cells had no available data or very few. This made it difficult to understand.

In [ ]:
df_weather_1 = pd.read_csv("new york city 2018-06-01 to 2018-08-31.csv")
df_weather_2 = pd.read_csv("new york city 2020-06-01 to 2020-08-31.csv")
df_weather = pd.concat([df_weather_1,df_weather_2],axis=0)
print(df_weather.co
df_weather.head()


In [ ]:
data = df.copy()
weather_cond = df_weather[['datetime','conditions']].copy()
weather_cond['datetime'] = pd.to_datetime(weather_cond['datetime'], format='%Y-%m-%d')

# Convert 'date' column in df to the same timezone as 'datetime' column in weather_cond
data['date'] = pd.to_datetime(pd.to_datetime(data['CRASH DATE']).dt.date)

# Merge weather conditions with accidents using pd.concat
data = data.merge(weather_cond, left_on='date', right_on='datetime', how='inner')
data.head()

In [ ]:
# calculate the mean of accidents per day and the mean for each conditions
# calculate overall mean
per_day = data[['date','conditions','CRASH TIME']].groupby(['date']).count().reset_index()
mean = per_day['CRASH TIME'].mean()

# mean per conditions
per_day_cond = data[['date','conditions','CRASH TIME']].groupby(['date','conditions']).count().reset_index()
mean_cond = per_day_cond[['conditions','CRASH TIME']].groupby(['conditions']).mean().reset_index()
mean_cond.columns = ['conditions','mean_cond']
# calcualte difference

mean_cond["diff"]=mean_cond["mean_cond"].apply(lambda x: x-mean)


In [ ]:
alt.Chart(mean_cond).mark_bar(height=3, orient='horizontal').encode(
    y=alt.Y('conditions:N').sort('x'),
    x='diff:Q',
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
).properties(
    width=500,
    height=300
)+alt.Chart(mean_cond).mark_point(orient='horizontal',size=100,opacity=1,fillOpacity=1).encode(
    y=alt.Y('conditions:N').sort('x'),
    x='diff:Q',
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    ),
    fill=alt.condition(
        alt.datum.diff > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
).properties(
    width=500,
    height=300
)

In [ ]:
def create_heatmap(df,weather_df,col_1,col_2,bins_1,bins_2,labels_1,labels_2):
    # Convert 'date' columns to datetime if they are not already in datetime format
    weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
    df['CRASH DATE'] = pd.to_datetime(pd.to_datetime(df['CRASH DATE']).dt.date)

    # Binarize temperature and precip columns
    weather_df['col_1'] = pd.cut(weather_df[col_1], bins=bins_1, labels=labels_1)
    weather_df['col_2'] = pd.cut(weather_df[col_2], bins=bins_2, labels=labels_2)
    # Merge the datasets on the date/datetime column
    merged_data= pd.merge(df, weather_df, left_on='CRASH DATE', right_on='datetime', how='inner')
    weather_count = weather_df.groupby(["col_1","col_2"]).size().reset_index(name='w_counts')
    # merge weather count and merged data on weather comb
    accident_count = merged_data.groupby(["col_1","col_2"]).size().reset_index(name='a_counts')
    accident_count = accident_count[accident_count["a_counts"]>0]
    data = pd.merge(weather_count, accident_count, on=['col_1', 'col_2'], how='inner')
    return alt.Chart(data).mark_rect().transform_calculate(
        value = 'datum.a_counts/datum.w_counts'
    ).encode(
        x=alt.X('col_1:O', title=col_1, sort=labels_1),
        y=alt.Y('col_2:O', title=col_2, sort=labels_2).sort("-y"),
        color = alt.Color('value:Q', title='Ratio'),
    ).properties(
        width=400
    )
def generate_bin_labels(bins):
    bin_labels = []
    for i, (start, end) in enumerate(zip(bins[:-1], bins[1:]), start=1):
        if end == float('inf'):
            bin_labels.append(f'{i}. {start}+')
        else:
            bin_labels.append(f'{i}. {start}-{end}')
    return bin_labels
import numpy as np
def generate_uniform_bins(dataframe_column, num_bins):
    min_val = dataframe_column.min()
    max_val = dataframe_column.max()
    bins = np.linspace(min_val, max_val, num=num_bins+1).tolist()
    return bins


In [ ]:
precip_bins = [0,2,5,7,10,15,100]
temp_bins = [15,20, 22.5,25,30,32,37]
temp_labels = generate_bin_labels(temp_bins)
precip_labels = generate_bin_labels(precip_bins)

map1=create_heatmap(df,df_weather,'temp','precip',temp_bins,precip_bins,temp_labels,precip_labels)

wind_bins = generate_uniform_bins(df_weather["windspeed"],4)
precip_bins = [0,2,5,7,10,15,100]
wind_labels = generate_bin_labels(wind_bins)
precip_labels = generate_bin_labels(precip_bins)

map2=create_heatmap(df,df_weather,'windspeed','precip',wind_bins,precip_bins,wind_labels,precip_labels)


In [ ]:
alt.hconcat(
    map1,
    map2,
    spacing=20  # Adjust the spacing between the heatmaps
).resolve_scale(
    y='shared'
)

## Results

FALTA FER DESCRIPCIO

In [ ]:
from graphs import *

**IMPORTANT** due to altair rendering issues, the following cells might require being run twice to display the graphs

In [ ]:
accident_data = get_accident_data("dataset_v1.csv", sample=False)
chart_1 = get_chart_1(accident_data)
chart_1

In [ ]:
create_chart2(q2_preprocessing(accident_data),height=500,width=300)


In [ ]:
from graphs import *
%load_ext autoreload
%reload_ext autoreload

In [ ]:
data_3 = q3_preprocessing(accident_data)
chart_3 = create_chart3(data_3,color_palette=get_palette())
chart_3

The visualizationa allows us to see wether some areas of the city have a higher concentration of accidents than others. The use of the hexagons allows for a detailed vision of the distribution of the accidents. The burough borders give, from a perception point of view, a feeling of containment within the buroughs and together with the burough labels allow us to orient ourselves around the map. 
Furthermore, the use of a sequential color palette clearly shows wether some areas have a higher count of accidents.

Nevertheless, the graph also has some issues. The main one is due to the nature of the data; the color palette has to deal with a big range of amounts as the quanity of accidents in some areas is much greater than in others. This makes differentiating at the lower and higher ends of the scale hard.
Furthermore, the burough labels are a bit hard to distinguish as they overlap the hexagons of the map. It would be better to manually situate them around the map with an arrow pointing to the burough. However, this task would be better solved by modifying the graph with a design software and not within altair.

Overall the chart allows the reader to compare different areas of the city and notice how areas such as manhattan or brookly have a much higher amount of accidents than other less trafficed areas.

In [ ]:
mapa = get_map()
ny_df, bur = get_buroughs(mapa)

hex_data = calculate_spatial_data(accident_data, mapa)
map_chart=plot_map(hex_data,mapa,ny_df,bur)
map_chart

Finally, we must also analyse the visualization which answers the final question; wether the weather has an effect on the amount of accidents. As can be seen in the following graph, we used a lollipop graph displaying for each weather condition the difference (in percentage) of the mean daily accidents in a weather condition when compared to the overall mean.

The visualization allows the reader to quickly observe that with bad weather the mean amount of accidents is greater than with good weather. The use of colors and the ordering allows to quickly distinguish between positive and negative differences and the use of the lollipop chart allows to obtain the value with detail.

However, one criticisim of this graph is that it hides further details about the distirbution of the data and, more importantly, it does not allow the reader to know the amount of days that each weather condition ocurred, which could skew the data. Nervertheless, as has been explained in the process section we felt that this was a sacrifice that had to be made in order to be able to have a clearer and more easy to understand visualization.

In [ ]:
weather_data = get_weather_data(accident_data)
weather_chart(weather_data)